# Visualizing Google GOES data (for research)
#### **Audience:** Anybody with a computer and access to at least 4GB of memory.
#### **Intent:** Provide a foundation for using GOES-R data in scientific research.

This notebook is an introduction to accessing GOES-R (GOES-16, GOES-17, and GOES-18) satellite data from the Google Cloud. It contains definitions to facilitate plotting, and provide a baseline for the analysis of, select GOES-R data products. Coding experience is required to use this notebook, and it is meant to be altered to fit a researcher's data interests. 

# Importing

In [ ]:
import ipywidgets as widgets
import requests
import netCDF4
from google.cloud import storage
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import cartopy.crs as ccrs
from cartopy import feature as cf
import xarray
from datetime import datetime

# Defining
#### Pre-written definitions to facilitate the visualization of select satellite data products. 

In [ ]:
def google_plot(satellite, product, fnames, urls):        
                
            # Aerosol Detection! 
            if product[:-1] == "ABI-L2-ADP":
                dat = dataset.metpy.parse_cf('Smoke')
                
                # Three variables to choose from
                smo = dataset['Smoke'].data
                dus = dataset['Dust'].data
                aer = dataset['Aerosol'].data
                
                # Identify plotting variable 
                plotting_var = smo
                
                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Identify mask options
                label_list = ['No smoke','Smoke']
            
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                cmap = plt.get_cmap('Set3', 2)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, cmap = cmap)
                
                cbar = plt.colorbar(cmap = cmap)
                cbar.set_ticks([])
                for j, lab in enumerate(label_list):
                    cbar.ax.text(1.5, (2 * j + 1) / (len(label_list)*2), lab, ha='left', va='baseline')   
                                       
                title = "Smoke Detection"
            
            # Cloud Top Height!
            elif product[:-1] == "ABI-L2-ACHA":
                dat = dataset.metpy.parse_cf('HT')
                
                # Identify variable 
                plotting_var = dataset['HT'].data

                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)   
                
                units = dataset['HT'].units
                plt.colorbar(cmap = "jet", label = units)
                
                title = "Cloud Top Height"
                
            # Cloud Top Temperature!
            elif product[:-1] == "ABI-L2-ACHT":
                dat = dataset.metpy.parse_cf('TEMP')
                
                # Identify variable and transform to Celsius
                plotting_var = dataset['TEMP'].data - 273.15

                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)   
                
                units = "Celsius"
                plt.colorbar(cmap = "jet", label = units)
                
                title = "Cloud Top \nTemperature"
                
            # Land Surface Temperature!
            elif product[:-1] == "ABI-L2-LST":
                dat = dataset.metpy.parse_cf('LST')
                
                # Identify variable and transform to Celsius
                plotting_var = dataset['LST'].data - 273.15
                
                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)   
                
                units = "Celsius"
                plt.colorbar(cmap = "jet", label = units)
                
                title = "Land Surface Temperature"
 
            # Lightning Detection!
            elif product == "GLM-L2-LCFA":
                dat = dataset.metpy.parse_cf('flash_energy')
                
                # Identify variable
                flash_energy = dataset['flash_energy'].data 
                
                # Identify plotting coordinates
                x_flash = dat.flash_lat
                y_flash = dat.flash_lon
                geos = dat.metpy.cartopy_crs  
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                lightning = plt.scatter(y_flash, x_flash,c = flash_energy, s = 5, label = "Flash Energy")
                
                units = dataset['flash_energy'].units 
                plt.colorbar(lightning, label = units)
                
                title = "Lightning Detection"
                
            # Rainfall Rate!
            elif product[:-1] == "ABI-L2-RRQPEF":
                dat = dataset.metpy.parse_cf('RRQPE')
                
                # Identify variable
                plotting_var = dataset['RRQPE'].data 

                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper',vmin=0, vmax=20, extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)  
                
                units = dataset['RRQPE'].units
                plt.colorbar(cmap = "jet", label = units)
                
                title = "Rainfall Rate: \nQuantitative Precipitation \nEstimate"
    
            # True color data!
            elif product[:-1] == "ABI-L2-MCMIP": 
                dat = dataset.metpy.parse_cf('CMI_C02')
                gamma = 2.2
                r = dataset['CMI_C02'].data; r = np.clip(r, 0, 1); r = np.power(r, 1/gamma)
                g = dataset['CMI_C03'].data; g = np.clip(g, 0, 1); g = np.power(g, 1/gamma)
                b = dataset['CMI_C01'].data; b = np.clip(b, 0, 1); b = np.power(b, 1/gamma)

                # Calculate the "True" Green
                g_true = 0.45 * r + 0.1 * g + 0.45 * b
                g_true = np.clip(g_true, 0, 1) 
                
                # Create a single RGB image for plotting
                plotting_var = np.dstack((r, g_true, b))
                
                # Identify plotting coordinates
                x = dat.x
                y = dat.y
                geos = dat.metpy.cartopy_crs
                
                # Plotting
                fig = plt.figure(figsize=(12, 7), dpi=100)
                ax = plt.axes(projection=geos)
                plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos)  
                
                title = "True Color Data"
                  
            nc.close()

            # For all plots
            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
    
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            
            plt.title(f"GOES-{satellite[-2:]} {title}", loc='left', fontweight='bold')
            plt.title('{}'.format(scan_start.strftime('%d %B %Y at %H:%M UTC')), loc='right')

            plt.show()
            plt.close(fig)
            plt.close("all")

In [ ]:
def google_animate(satellite, product):

    # Lightning Detection!
    if product == "GLM-L2-LCFA":
        
        # Reading in true color data
        fnames_truecolor = []
        urls_truecolor = []
        for hour in range(int(shour), int(ehour)+1):
            prefix = (f"ABI-L2-MCMIPM/{year}/{doy}/{hour}/")
            blobs = bucket.list_blobs(prefix = prefix, delimiter = "/")
            response = blobs._get_next_page_response()

            # Refine data files to chosen minute(s)
            for i in range(0, len(response["items"])):
                url_link = ((response["items"])[i])["mediaLink"]
                file_name = ((response["items"])[i])["name"]

                file_start = file_name.split('_')[-3][8:12]
                file_end = file_name.split('_')[-2][8:12]

                if ((int(file_start[0:4]) >= int((str(shour) + str(smin)))) and (int(file_end[0:4]) <= int((str(ehour) + str(emin))))):
                    urls_truecolor.append(url_link)
                    fnames_truecolor.append(file_name)  

        fig, ax = plt.subplots()

        def animate(i):    
            if len(fnames_truecolor) > 1:
                fname_truecolor = (fnames_truecolor[(round((len(fnames_truecolor)/len(fnames))*i))])[12:]
                url_truecolor = (urls_truecolor[(round((len(urls_truecolor)/len(urls))*i))])
            else:
                fname_truecolor = (fnames_truecolor[0])[12:]
                url_truecolor = urls_truecolor[0]
            
            resp_truecolor = requests.get(url_truecolor)
            nc_truecolor = netCDF4.Dataset(fname_truecolor, memory = resp_truecolor.content)
            
            dataset_truecolor = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc_truecolor))
            dataset_truecolor.load()
            dat_truecolor = dataset_truecolor.metpy.parse_cf('CMI_C02')
            gamma = 2.2
            r = dataset_truecolor['CMI_C02'].data; r = np.clip(r, 0, 1); r = np.power(r, 1/gamma)
            g = dataset_truecolor['CMI_C03'].data; g = np.clip(g, 0, 1); g = np.power(g, 1/gamma)
            b = dataset_truecolor['CMI_C01'].data; b = np.clip(b, 0, 1); b = np.power(b, 1/gamma)

            # Calculate the "True" Green
            g_true = 0.45 * r + 0.1 * g + 0.45 * b
            g_true = np.clip(g_true, 0, 1) 
            plotting_var = np.dstack((r, g_true, b))

            # Identify plotting coordinates
            x_truecolor = dat_truecolor.x
            y_truecolor = dat_truecolor.y

            geos_truecolor = dat_truecolor.metpy.cartopy_crs

            fig.clear()
            ax = fig.add_subplot(projection=geos_truecolor)

            #lightning data 
            fname = (fnames[i])[12:]
            resp = requests.get(urls[i])
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('flash_energy')
            x = dat.flash_lat
            y = dat.flash_lon
            geos = dat.metpy.cartopy_crs 
            
            dataset.close()
            dataset_truecolor.close()

            plt.imshow(plotting_var, origin='upper', extent=(x_truecolor.min(), x_truecolor.max(), y_truecolor.min(), y_truecolor.max()), transform=geos_truecolor)
            ax.scatter(y, x, c = "blue", s=5, transform=geos)

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x_truecolor.min(), x_truecolor.max(), y_truecolor.min(), y_truecolor.max()], geos_truecolor)
            
            title = "Lightning Detection"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')

        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames)-1, interval=200)
        ani.save('GOES_animation.gif', writer='pillow')
        plt.close("all")
        
    # Land Surface Temperature!
    elif product[:-1] == "ABI-L2-LST":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(urls[i])
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('LST')
                
            # Identify variable and transform to Celsius
            plotting_var = dataset['LST'].data - 273.15
            # Identify plotting coordinates
            x = dat.x
            y = dat.y

            geos = dat.metpy.cartopy_crs
            
            dataset.close()

            fig.clear()
            ax = fig.add_subplot(projection=geos)
            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=-40, vmax=40)

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            title = "Land Surface Temperature"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
            plt.colorbar(label = "Celsius")
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames)-1, interval=200)
        ani.save('GOES_LST_animation.gif', writer='pillow')
        plt.close("all")
        plt.close(fig)
        
    # Cloud Top Temperature!
    elif product[:-1] == "ABI-L2-ACHT":
        
        fig, ax = plt.subplots()
        
        def animate(i):    
            fname = (fnames[i])[12:]
            resp = requests.get(urls[i])
            nc = netCDF4.Dataset(fname, memory = resp.content)
            dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
            dataset.load()
            dat = dataset.metpy.parse_cf('TEMP')
                
            # Identify variable and transform to Celsius
            plotting_var = dataset['TEMP'].data - 273.15
            # Identify plotting coordinates
            x = dat.x
            y = dat.y

            geos = dat.metpy.cartopy_crs

            dataset.close()
            
            fig.clear()
            ax = fig.add_subplot(projection=geos)

            plt.imshow(plotting_var, origin='upper', extent=(x.min(), x.max(), y.min(), y.max()), transform=geos, vmin=-80, vmax=20)

            ax.coastlines(resolution='50m', color='black', linewidth=0.25)
            ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
            ax.set_extent([x.min(), x.max(), y.min(), y.max()], geos)
            title = "Cloud Top Temperature"
            scan_start = datetime.strptime(dataset.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')
            plt.title(f"GOES-{satellite[-2:]} {title} \n{scan_start.strftime('%d %B %Y at %H:%M UTC')}", loc='left', fontweight='bold')
            plt.colorbar(label = "Celsius")
            
        ani = animation.FuncAnimation(fig, animate, repeat=True, frames=len(fnames)-1, interval=200)
        ani.save('GOES_CTT_animation.gif', writer='pillow')
        plt.close("all")
        plt.close(fig)

# Satellite parameters!
#### The option to customize your data search.

In [ ]:
# Choose a satellite
satellite = "gcp-public-data-goes-16" # options are 'gcp-public-data-goes-16', 'gcp-public-data-goes-17', or 'gcp-public-data-goes-18'

"""
# Explore Google's available products:
from google.cloud import storage

products = []

client = storage.Client.create_anonymous_client()
bucket = client.bucket(bucket_name = satellite)
blobs = bucket.list_blobs(delimiter = "/")
response = blobs._get_next_page_response()
for product in response['prefixes']:
    print(product)
"""

# Choose a product
product = "ABI-L2-ACHTM" # the last letter denotes scan type (Full Disk, CONUS/PACUS, or Meso)

# Choose a date
year = "2023" # must be 4 characters (i.e. "2022")
doy = "242" # must be 3 characters (i.e. "001")

# Choose a time
shour = "16" # must be 2 characters (i.e. "01")
smin = "00" # must be 2 characters (i.e. "01")

ehour = "16" # must be 2 characters (i.e. "01")
emin = "10" # must be 2 characters (i.e. "01")

client = storage.Client.create_anonymous_client()
bucket = client.bucket(bucket_name = satellite)

fnames = []
urls = []
for hour in range(int(shour), int(ehour)+1):
    prefix = (f"{product}/{year}/{doy}/{hour}/")
    blobs = bucket.list_blobs(prefix = prefix, delimiter = "/")
    response = blobs._get_next_page_response()

    # Refine data files to chosen minute(s)
    for i in range(0, len(response["items"])):
        url_link = ((response["items"])[i])["mediaLink"]
        file_name = ((response["items"])[i])["name"]

        file_start = file_name.split('_')[-3][8:12]
        file_end = file_name.split('_')[-2][8:12]
            
        if ((int(file_start[0:4]) >= int((str(shour) + str(smin)))) and (int(file_end[0:4]) <= int((str(ehour) + str(emin))))):
            urls.append(url_link)
            fnames.append(file_name)  
        
print(f"This search resulted in {len(fnames)} data files.")

# Visualize the data!

In [ ]:
# Plot single frame(s) of your data

for f in range(0, 1):  # just plotting the first image of your search
# for f in range(0, len(fnames)): # plotting all searched images

    # Retrieve files from URL
    resp = requests.get(urls[f])
    
    # Read in with netCDF4 and xarray
    nc = netCDF4.Dataset(fnames[f], memory = resp.content)
    dataset = xarray.open_dataset(xarray.backends.NetCDF4DataStore(nc))
    dataset.load()
    
    # Visualize the data
    google_plot(satellite, product, fnames, urls)

In [ ]:
# Plot an animation of all searched frames

google_animate(satellite, product)

In [ ]:
# Your analysis here!

**GOES Product Documentation:**
- https://www.goes-r.gov/products/overview.html (Broad explanation of all products)
- https://www.star.nesdis.noaa.gov/goesr/ (Algorithm & product description)
- http://cimss.ssec.wisc.edu/goes/OCLOFactSheetPDFs/ (Quick guides)
- https://github.com/awslabs/open-data-docs/blob/main/docs/noaa/noaa-goes16/README.md (Variables & product names) 
- https://www.goes-r.gov/products/docs/PUG-L2+-vol5.pdf (Technical explanation of each product and variable)

**Coding Sources:**
- https://unidata.github.io/python-gallery/examples/mapping_GOES16_TrueColor.html#sphx-glr-download-examples-mapping-goes16truecolor-py   
- https://unidata.github.io/python-training/gallery/mapping_goes16_truecolor/   
- https://fire.trainhub.eumetsat.int/docs/figure3_GOES17_L1B_and_L2.html   
- https://unidata.github.io/python-training/gallery/mapping_goes16_truecolor/

**CSP Access:**    
AWS: https://registry.opendata.aws/noaa-goes/   
Google: https://console.cloud.google.com/marketplace/product/noaa-public/goes   
Azure: https://microsoft.github.io/AIforEarthDataSets/data/goes-r.html   